# PEDP Network - Hypothetical Future State (Scenario Analysis)

⚠️ **THIS IS NOT THE CURRENT NETWORK - SCENARIO ANALYSIS ONLY** ⚠️

This notebook creates a **hypothetical future state** visualization showing how the isolated clusters could become connected through intermediary organizations.

**Hypothetical Elements (NOT REAL):**
- **2 new intermediary nodes** (box-shaped, grey borders):
  - Regional Data Coordination Hub (HYP-HUB1)
  - Funder Collaborative Network (HYP-HUB2)
- **~50 grey edges** connecting isolated organizations through these hubs

**What's Preserved from Current Network:**
- All 78 existing nodes in exact same positions
- All 55 existing relationships (purple, green, blue edges)
- Node sizes calculated from real connections only

**Purpose:**
This visualization helps stakeholders:
1. See what a more connected ecosystem could look like
2. Identify strategic opportunities for intermediary organizations
3. Understand pathways to engage isolated funders and practitioners

**Compare with Current Network:**
Open `network_map.html` side-by-side to see the contrast between current (fragmented) and hypothetical (connected) states.

## Section 1: Setup

In [1]:
import networkx as nx
import pandas as pd
from pyvis.network import Network
import warnings
warnings.filterwarnings('ignore')

# Edge styling by relationship type (includes HYPOTHETICAL)
EDGE_STYLES = {
    'is a member of': {'color': '#8e44ad', 'width': 2.5, 'arrows': 'to'},
    'funds': {'color': '#27ae60', 'width': 3, 'arrows': 'to'},
    'coordinates action with': {'color': '#3498db', 'width': 2, 'arrows': 'to;from'},
    'hypothetical connection': {'color': '#999999', 'width': 1.5, 'arrows': 'to', 'dashes': True}  # NEW
}

print("✓ Libraries imported successfully")
print(f"✓ NetworkX version: {nx.__version__}")
print(f"✓ Pandas version: {pd.__version__}")
print("\n⚠️  HYPOTHETICAL FUTURE STATE MODE - Scenario Analysis")

✓ Libraries imported successfully
✓ NetworkX version: 3.5
✓ Pandas version: 2.2.2

⚠️  HYPOTHETICAL FUTURE STATE MODE - Scenario Analysis


## Section 2: Load Data (Current + Hypothetical)

In [2]:
# Load BOTH current and hypothetical data
print("Loading CURRENT network data...")
nodes_current = pd.read_csv('../data/processed/nodes.csv')
edges_current = pd.read_csv('../data/processed/edges.csv')
positions_current = pd.read_csv('../data/processed/node_positions.csv')

print("Loading HYPOTHETICAL additions...")
nodes_hyp = pd.read_csv('../data/processed/nodes_hypothetical.csv')
edges_hyp = pd.read_csv('../data/processed/edges_hypothetical.csv')
positions_hyp = pd.read_csv('../data/processed/node_positions_hypothetical.csv')

# Combine current + hypothetical
nodes_df = pd.concat([nodes_current, nodes_hyp], ignore_index=True)
edges_df = pd.concat([edges_current, edges_hyp], ignore_index=True)
positions_df = pd.concat([positions_current, positions_hyp], ignore_index=True)

# Load color config and create mapping
colors_df = pd.read_csv('../data/processed/colors.csv')
color_map = dict(zip(colors_df['name'], colors_df['hex']))

# Map color names to hex codes
nodes_df['hex_color'] = nodes_df['color'].map(color_map)

# Create positions mapping for quick lookup
positions_map = {row['id']: {'x': row['x'], 'y': row['y'], 'fixed': row['fixed']}
                 for _, row in positions_df.iterrows()}

# Display summary
print(f"\nCombined Network: {len(nodes_df)} nodes, {len(edges_df)} edges")
print(f"  - Current nodes: {len(nodes_current)}, Hypothetical nodes: {len(nodes_hyp)}")
print(f"  - Current edges: {len(edges_current)}, Hypothetical edges: {len(edges_hyp)}")

print("\n=== Hypothetical Nodes ===  ")
display(nodes_hyp[['id', 'name', 'category', 'description']])

print("\n=== Sample Hypothetical Edges ===")
display(edges_hyp.head(10))

print("\n=== Relationship Type Distribution ===  ")
print(edges_df['relationship_type'].value_counts())

Loading CURRENT network data...
Loading HYPOTHETICAL additions...

Combined Network: 80 nodes, 105 edges
  - Current nodes: 78, Hypothetical nodes: 2
  - Current edges: 55, Hypothetical edges: 50

=== Hypothetical Nodes ===  


,id,name,category,description
0,HYP-HUB1,Regional Data Coordination Hub,Data Coordination/Standards,[HYPOTHETICAL] Regional organization coordinat...
1,HYP-HUB2,Funder Collaborative Network,Funder,[HYPOTHETICAL] Funder collaborative pooling re...



=== Sample Hypothetical Edges ===


,source,target,relationship_type
0,SloanFoundation,HYP-HUB2,hypothetical connection
1,GatesFoundation,HYP-HUB2,hypothetical connection
2,HewlettFoundation,HYP-HUB2,hypothetical connection
3,OpenDataPolicyLab,HYP-HUB2,hypothetical connection
4,ResourcesLegacyFund,HYP-HUB2,hypothetical connection
5,Googleorg,HYP-HUB2,hypothetical connection
6,NavigationFund,HYP-HUB2,hypothetical connection
7,PiscesFoundation,HYP-HUB2,hypothetical connection
8,QCF,HYP-HUB2,hypothetical connection
9,RennaissanceFoundati,HYP-HUB2,hypothetical connection



=== Relationship Type Distribution ===  
relationship_type
hypothetical connection    50
coordinates action with    33
funds                      18
is a member of              4
Name: count, dtype: int64


## Section 3: Build Network (with REAL and HYPOTHETICAL graphs)

In [3]:
# Build FULL graph (including hypothetical edges for visualization)
G = nx.DiGraph()

# Add nodes with attributes
for idx, row in nodes_df.iterrows():
    G.add_node(
        row['id'],
        name=row['name'],
        organization=row['organization'],
        category=row['category'],
        description=row['description'],
        status=row['status'],
        timeline=row['timeline']
    )

# Add edges with relationship type attribute
for idx, row in edges_df.iterrows():
    G.add_edge(
        row['source'],
        row['target'],
        relationship_type=row['relationship_type']
    )

# Build REAL graph (excluding hypothetical edges for metrics)
print("Building REAL graph (excluding hypothetical edges)...")
G_real = nx.DiGraph()
G_real.add_nodes_from(G.nodes(data=True))

for source, target, data in G.edges(data=True):
    if data['relationship_type'] != 'hypothetical connection':
        G_real.add_edge(source, target, **data)

# Convert to undirected for connectivity checks
G_undirected = G.to_undirected()
G_real_undirected = G_real.to_undirected()

# Network statistics - FULL graph (with hypothetical)
print("\n=== HYPOTHETICAL Network Statistics (Full Graph) ===")
print(f"Nodes: {G.number_of_nodes()}")
print(f"Edges: {G.number_of_edges()}")
print(f"Density: {nx.density(G):.3f}")
print(f"Connected: {nx.is_connected(G_undirected)}")
print(f"Connected components: {nx.number_connected_components(G_undirected)}")

# Count isolated nodes in FULL graph
isolated_count_full = sum(1 for n in G.nodes() if G_undirected.degree(n) == 0)
print(f"Isolated nodes: {isolated_count_full}")

# Network statistics - REAL graph (without hypothetical)
print("\n=== REAL Network Statistics (Excluding Hypothetical) ===")
print(f"Nodes: {G_real.number_of_nodes()}")
print(f"Edges: {G_real.number_of_edges()}")
print(f"Connected: {nx.is_connected(G_real_undirected)}")
print(f"Connected components: {nx.number_connected_components(G_real_undirected)}")

# Count isolated nodes in REAL graph
isolated_count_real = sum(1 for n in G_real.nodes() if G_real_undirected.degree(n) == 0)
print(f"Isolated nodes: {isolated_count_real}")

# Show relationship type distribution
print("\n=== Relationship Type Distribution ===")
print(edges_df['relationship_type'].value_counts())

Building REAL graph (excluding hypothetical edges)...

=== HYPOTHETICAL Network Statistics (Full Graph) ===
Nodes: 80
Edges: 105
Density: 0.017
Connected: True
Connected components: 1
Isolated nodes: 0

=== REAL Network Statistics (Excluding Hypothetical) ===
Nodes: 80
Edges: 55
Connected: False
Connected components: 49
Isolated nodes: 48

=== Relationship Type Distribution ===
relationship_type
hypothetical connection    50
coordinates action with    33
funds                      18
is a member of              4
Name: count, dtype: int64


## Section 4: Calculate Centrality Metrics (REAL edges only)

In [4]:
# CRITICAL: Calculate centrality on REAL graph only (preserves original node sizes)
print("Calculating centrality metrics on REAL edges only...")

# Create filtered REAL graph for node sizing (exclude funder network edges)
G_real_filtered = nx.DiGraph()
G_real_filtered.add_nodes_from(G_real.nodes(data=True))

for source, target, data in G_real.edges(data=True):
    if data['relationship_type'] != 'Interested in solving the problem':
        G_real_filtered.add_edge(source, target, **data)

G_real_filtered_undirected = G_real_filtered.to_undirected()

# Calculate centrality on REAL filtered graph for node sizing
degree_centrality_for_sizing = nx.degree_centrality(G_real_filtered_undirected)

# Calculate other centrality metrics on REAL full graph for statistics
degree_centrality = nx.degree_centrality(G_real_undirected)
betweenness_centrality = nx.betweenness_centrality(G_real_undirected)
closeness_centrality = nx.closeness_centrality(G_real_undirected)

# Create summary DataFrame using REAL metrics
centrality_data = []
for node in G.nodes():
    # Skip hypothetical nodes in centrality calculations
    if node.startswith('HYP-'):
        continue
        
    node_name = nodes_df[nodes_df['id'] == node]['name'].values[0]
    centrality_data.append({
        'ID': node,
        'Node': node_name,
        'Connections': G_real_undirected.degree(node),
        'Meaningful_Connections': G_real_filtered_undirected.degree(node),
        'Degree': degree_centrality[node],
        'Betweenness': betweenness_centrality[node],
        'Closeness': closeness_centrality[node]
    })

centrality_df = pd.DataFrame(centrality_data).sort_values('Degree', ascending=False)

print("\n=== Top 10 Most Connected (by REAL edges) ===")
display(centrality_df.head(10))

# Verify PEDP centrality
pedp_real = G_real_undirected.degree('PEDP')
pedp_full = G_undirected.degree('PEDP')
print(f"\n✓ PEDP Verification:")
print(f"  - REAL connections: {pedp_real}")
print(f"  - FULL connections (with hypothetical): {pedp_full}")
print(f"  - Node size will be based on {pedp_real} REAL connections (not {pedp_full})")

Calculating centrality metrics on REAL edges only...

=== Top 10 Most Connected (by REAL edges) ===


,ID,Node,Connections,Meaningful_Connections,Degree,Betweenness,Closeness
15,PEDP,Public Environmental Data Partners,27,27,0.341772,0.128308,0.347559
2,DataFoundation,Data Foundation - Climate Data Collaborative &...,9,9,0.113924,0.020334,0.229520
5,NASEM,NASEM - Earth Observations & Data Stewardship ...,8,8,0.101266,0.012745,0.225270
12,CODE,CODE - Center for Open Data Enterprise,6,6,0.075949,0.002039,0.213413
0,AGU,American Geophysical Union,6,6,0.075949,0.003673,0.217224
3,GRQD,Group on Reference Quality Datasets,5,5,0.063291,0.000974,0.160060
9,NYCE,New York Climate Exchange,4,4,0.050633,0.000757,0.209734
10,DRP,The Data Rescue Project,4,4,0.050633,0.000325,0.196203
4,KCF,Keeling Curve Foundation,3,3,0.037975,0.000000,0.153982
17,EPIC,Environmental Policy Innovation Center,3,3,0.037975,0.000000,0.202743



✓ PEDP Verification:
  - REAL connections: 27
  - FULL connections (with hypothetical): 29
  - Node size will be based on 27 REAL connections (not 29)


## Section 5: Create Interactive Visualization (Hypothetical Future State)

In [5]:
# Initialize PyVis network with LIGHT GREY background for hypothetical state
net = Network(
    height='800px',
    width='100%',
    bgcolor='#f8f8f8',  # Light grey background
    font_color='#333333',
    notebook=True,
    directed=True
)

# CRITICAL: MINIMAL physics to preserve existing layout
# Much weaker forces than current network to avoid moving nodes
net.barnes_hut(
    gravity=-500,           # MUCH weaker than current (-3000)
    central_gravity=0.01,   # Nearly zero (vs 0.1)
    spring_length=150,      # Same as current
    spring_strength=0.001,  # 10x weaker than current (0.01)
    damping=0.9,            # High damping to prevent movement (vs 0.2)
    overlap=0
)

# Add nodes with styling
for node in G.nodes():
    node_data = nodes_df[nodes_df['id'] == node].iloc[0]
    color = node_data['hex_color']
    
    # Get position from positions_map
    pos = positions_map[node]
    
    # Check if hypothetical node
    is_hypothetical = node.startswith('HYP-')
    
    # Build tooltip (same logic as current network)
    tooltip_lines = [node_data['name'], f"Category: {node_data['category']}", ""]
    
    if is_hypothetical:
        tooltip_lines.append("⚠️ HYPOTHETICAL ORGANIZATION (NOT REAL)")
        tooltip_lines.append("")
        tooltip_lines.append(node_data['description'])
    else:
        # Real node - check connections
        node_degree = G_real_undirected.degree(node)
        
        if node_degree == 0:
            if node_data['category'] == 'Funder':
                tooltip_lines.append("Interested in working in this space")
            else:
                tooltip_lines.append("Actively working in this space")
        else:
            # Show REAL connections only in tooltip
            connections = {
                'member_of': [],
                'has_members': [],
                'funds': [],
                'funded_by': [],
                'coordinates': []
            }
            
            # Process REAL outgoing edges only
            for _, target, edge_data in G_real.out_edges(node, data=True):
                rel_type = edge_data['relationship_type']
                target_name = G_real.nodes[target]['name']
                
                if rel_type == "is a member of":
                    connections['member_of'].append(target_name)
                elif rel_type == "funds":
                    connections['funds'].append(target_name)
                elif rel_type == "coordinates action with":
                    connections['coordinates'].append(target_name)
            
            # Process REAL incoming edges only
            for source, _, edge_data in G_real.in_edges(node, data=True):
                rel_type = edge_data['relationship_type']
                source_name = G_real.nodes[source]['name']
                
                if rel_type == "is a member of":
                    connections['has_members'].append(source_name)
                elif rel_type == "funds":
                    connections['funded_by'].append(source_name)
                elif rel_type == "coordinates action with":
                    if source_name not in connections['coordinates']:
                        connections['coordinates'].append(source_name)
            
            # Build tooltip sections
            if connections['member_of']:
                tooltip_lines.append("Member of:")
                for org in sorted(connections['member_of']):
                    tooltip_lines.append(f"• {org}")
                tooltip_lines.append("")
            
            if connections['has_members']:
                tooltip_lines.append("Has members:")
                for org in sorted(connections['has_members']):
                    tooltip_lines.append(f"• {org}")
                tooltip_lines.append("")
            
            if connections['funds']:
                tooltip_lines.append("Funds:")
                for org in sorted(connections['funds']):
                    tooltip_lines.append(f"• {org}")
                tooltip_lines.append("")
            
            if connections['funded_by']:
                tooltip_lines.append("Funded by:")
                for org in sorted(connections['funded_by']):
                    tooltip_lines.append(f"• {org}")
                tooltip_lines.append("")
            
            if connections['coordinates']:
                tooltip_lines.append("Coordinates with:")
                for org in sorted(connections['coordinates']):
                    tooltip_lines.append(f"• {org}")
    
    title = "\n".join(tooltip_lines).rstrip()
    
    if is_hypothetical:
        # Hypothetical intermediary nodes - distinct styling
        net.add_node(
            node,
            label=f"[HYPOTHETICAL]\n{node_data['name']}",
            title=title,
            color=color,
            size=30,  # Fixed medium size
            borderWidth=3,
            borderWidthSelected=5,
            shape='box',  # Different shape
            font={'color': '#666666'},  # Grey text
            x=pos['x'],
            y=pos['y'],
            fixed=True  # ALWAYS fixed for hypothetical nodes
        )
    else:
        # Existing nodes - use REAL graph metrics for size, FORCE fixed=True
        if node_data['category'] == 'Funder':
            size = 20  # Fixed small size for all funders
        else:
            # Size by degree centrality from REAL edges only
            size = 15 + (degree_centrality_for_sizing.get(node, 0) * 200)
        
        net.add_node(
            node,
            label=node_data['name'],
            title=title,
            color=node_data['hex_color'],
            size=size,
            borderWidth=2,
            borderWidthSelected=4,
            x=pos['x'],
            y=pos['y'],
            fixed=True  # CRITICAL: Force ALL nodes to be fixed
        )

# Add styled edges based on relationship type
for edge in G.edges(data=True):
    rel_type = edge[2]['relationship_type']
    style = EDGE_STYLES[rel_type]
    
    edge_config = {
        'color': style['color'],
        'width': style['width'],
        'arrows': style['arrows'],
        'title': rel_type,
        'smooth': {'type': 'continuous'},
        'arrowStrikethrough': False
    }
    
    # Add dashes if specified
    if style.get('dashes'):
        edge_config['dashes'] = True
    
    net.add_edge(edge[0], edge[1], **edge_config)

# Show in notebook
print("Generating HYPOTHETICAL FUTURE STATE visualization...")
print("\n⚠️  CRITICAL: This is a scenario analysis, not the current network!")
print("\n💡 What's Different:")
print("   • 2 new intermediary nodes (box-shaped, grey)")
print("   • ~50 grey edges connecting isolated organizations")
print("   • All existing nodes/edges preserved in exact same positions")
print("   • Node sizes based on REAL connections only\n")
net.show('network_preview_hypothetical.html')

# Save to outputs directory
net.save_graph('../outputs/network_map_hypothetical.html')
print("\n✓ Hypothetical visualization saved to: outputs/network_map_hypothetical.html")
print("\n📊 Open side-by-side with network_map.html to compare current vs future states!")

Generating HYPOTHETICAL FUTURE STATE visualization...

⚠️  CRITICAL: This is a scenario analysis, not the current network!

💡 What's Different:
   • 2 new intermediary nodes (box-shaped, grey)
   • ~50 grey edges connecting isolated organizations
   • All existing nodes/edges preserved in exact same positions
   • Node sizes based on REAL connections only

network_preview_hypothetical.html

✓ Hypothetical visualization saved to: outputs/network_map_hypothetical.html

📊 Open side-by-side with network_map.html to compare current vs future states!


## Section 6: Network Summary (Hypothetical vs Current)

In [6]:
print("="*60)
print("PEDP CLIMATE & ENVIRONMENTAL DATA INITIATIVES")
print("Network Summary - HYPOTHETICAL FUTURE STATE")
print("⚠️  THIS IS NOT THE CURRENT NETWORK - SCENARIO ANALYSIS ⚠️")
print("="*60)

print("\n=== HYPOTHETICAL State (Full Graph) ===")
print(f"📊 Total Nodes: {G.number_of_nodes()} (78 current + 2 hypothetical)")
print(f"🔗 Total Edges: {G.number_of_edges()} (55 current + 50 hypothetical)")
print(f"🌐 Connected Components: {nx.number_connected_components(G_undirected)} (down from 47)")
print(f"⭐ Isolated Nodes: {isolated_count_full} (down from 46)")

print("\n=== CURRENT State (Real Edges Only) ===")
print(f"📊 Total Nodes: {G_real.number_of_nodes()}")
print(f"🔗 Total Edges: {G_real.number_of_edges()}")
print(f"🌐 Connected Components: {nx.number_connected_components(G_real_undirected)}")
print(f"⭐ Isolated Nodes: {isolated_count_real}")

print("\n=== Hypothetical Intermediary Nodes ===")
for idx, row in nodes_hyp.iterrows():
    print(f"\n{row['name']}:")
    print(f"  Category: {row['category']}")
    print(f"  Purpose: {row['description']}")
    
    # Count hypothetical connections
    in_degree = G.in_degree(row['id'])
    out_degree = G.out_degree(row['id'])
    print(f"  Connections: {in_degree} incoming, {out_degree} outgoing")

print("\n=== Relationship Type Distribution ===")
rel_counts = edges_df['relationship_type'].value_counts()
for rel_type, count in rel_counts.items():
    style = EDGE_STYLES[rel_type]
    symbol = "⚠️ " if rel_type == 'hypothetical connection' else "   "
    print(f"{symbol}{rel_type:30s} {count:3d} edges ({style['color']})")

print("\n=== Top 5 Key Hubs (by REAL connections) ===")
for idx, row in centrality_df.head(5).iterrows():
    print(f"{row['Node']:50s} {row['Connections']:2d} connections")

print("\n" + "="*60)
print("💡 Hypothetical Future State Insights:")
print("   - Network becomes fully connected (1 component vs 47)")
print("   - All 46 isolated organizations now engaged")
print("   - 2 intermediary hubs create pathways:")
print("     • HYP-HUB1: Coordinates practitioners/data initiatives")
print("     • HYP-HUB2: Coordinates funders/resources")
print("   - Grey edges show potential connections (NOT current)")
print("\n⚠️  To Create This Future State:")
print("   - Establish intermediary coordination organizations")
print("   - Develop engagement pathways for isolated funders")
print("   - Build coordination mechanisms for practitioners")
print("   - Strategic investment in network infrastructure")
print("="*60)

PEDP CLIMATE & ENVIRONMENTAL DATA INITIATIVES
Network Summary - HYPOTHETICAL FUTURE STATE
⚠️  THIS IS NOT THE CURRENT NETWORK - SCENARIO ANALYSIS ⚠️

=== HYPOTHETICAL State (Full Graph) ===
📊 Total Nodes: 80 (78 current + 2 hypothetical)
🔗 Total Edges: 105 (55 current + 50 hypothetical)
🌐 Connected Components: 1 (down from 47)
⭐ Isolated Nodes: 0 (down from 46)

=== CURRENT State (Real Edges Only) ===
📊 Total Nodes: 80
🔗 Total Edges: 55
🌐 Connected Components: 49
⭐ Isolated Nodes: 48

=== Hypothetical Intermediary Nodes ===

Regional Data Coordination Hub:
  Category: Data Coordination/Standards
  Purpose: [HYPOTHETICAL] Regional organization coordinating data initiatives and connecting funders with practitioners
  Connections: 2 incoming, 23 outgoing

Funder Collaborative Network:
  Category: Funder
  Purpose: [HYPOTHETICAL] Funder collaborative pooling resources and coordinating grants in the environmental data space
  Connections: 23 incoming, 2 outgoing

=== Relationship Type Distr

## Validation: Verify Layout Preservation

In [7]:
# CRITICAL: Verify node positions unchanged for existing nodes
print("Validating that existing node positions are preserved...\n")

current_pos_map = {row['id']: (row['x'], row['y']) 
                   for _, row in positions_current.iterrows()}

hypothetical_pos_map = {row['id']: (row['x'], row['y']) 
                        for _, row in positions_df.iterrows()
                        if not row['id'].startswith('HYP-')}

all_match = True
for node_id, (x, y) in current_pos_map.items():
    if node_id not in hypothetical_pos_map:
        print(f"❌ Missing node {node_id} in hypothetical positions")
        all_match = False
        continue
        
    hyp_x, hyp_y = hypothetical_pos_map[node_id]
    if x != hyp_x or y != hyp_y:
        print(f"❌ Node {node_id} position changed: ({x},{y}) -> ({hyp_x},{hyp_y})")
        all_match = False

if all_match:
    print("✅ All existing node positions preserved exactly")
else:
    print("\n⚠️  WARNING: Some node positions changed!")

# Verify node sizes based on real edges only
print("\nVerifying node sizes based on REAL edges only...")
pedp_degree_real = G_real_undirected.degree('PEDP')
pedp_degree_full = G_undirected.degree('PEDP')
print(f"✅ PEDP: {pedp_degree_real} real connections (size calculation)")
print(f"   vs {pedp_degree_full} total connections (including hypothetical)")
print(f"   → Node size based on {pedp_degree_real} connections (correct)")

print("\n" + "="*60)
print("✅ VALIDATION COMPLETE")
print("   • Existing node positions: PRESERVED")
print("   • Node sizes: Based on REAL edges only")
print("   • Hypothetical elements: Clearly marked")
print("="*60)

Validating that existing node positions are preserved...

✅ All existing node positions preserved exactly

Verifying node sizes based on REAL edges only...
✅ PEDP: 27 real connections (size calculation)
   vs 29 total connections (including hypothetical)
   → Node size based on 27 connections (correct)

✅ VALIDATION COMPLETE
   • Existing node positions: PRESERVED
   • Node sizes: Based on REAL edges only
   • Hypothetical elements: Clearly marked
